# Deriving Grammatical Gender Rule for German Nouns

This notebook serves as a basis of one of the services implemented in the open - source software for learning of German for English speakers. In this notebook, a set of rules are derived which allow with accuracy of 82% to determine the grammatical gender of German noun. Such rule applies to roughly 80% (6000+) of frequently used nouns in German. The repository is [here](https://gitlab.com/iaros/learn-german), and you can give the software in repository a try by navigating to [bit.do/d3l](bit.do/d3l); Here d3l stands for "Data Driven Deutsch Lernen".

## The why

So why care about grammatical gender of a German noun? Consider the following two phrases in German and their translation:

`Ich gebe den Vogel der Katze.` = "I give the bird to the cat"

`Ich gebe dem Vogel die Katze.` = "To the bird I give the cat "

See, the direction of action (who is given to who) is determined by articles (`den, der, dem, die`) in the sentences above; by switching the articles only, the direction is reversed. Imagine what kind of derps and misunderstandings an incorrect use of articles can lead to! "I gave the lady to the documents", "I received my landlord from keys", ...

To form a correct article before a noun (e.g. `den` before `Vogel` or `der` before `Katze`), you need to know a correct case of a noun, as well as the gender of the noun. For some information on how to determine a case of noun, see [here](https://www.quora.com/What-is-the-simplest-explanation-for-the-difference-between-nominative-accusative-dative-and-genitive-articles) or [here](http://www.lsa.umich.edu/german/hmr/grammatik/basic_chart.html). 

As for the second ingredient - gender of noun, it is even more complex; every noun in German language has a grammatical gender, one of `maskulin`, `feminin` and `neuter`. So should one learn genders for 30k+ German nouns!? No, not really. This would be a rather daunting task, so people tried to come up with some classification rules, which help to determine the gender of a noun. For example, a set of such rules is described in a [great article by fluent in 3 month](https://www.fluentin3months.com/german-noun-genders/). A classification of a noun is done by it's ending, with rules given below:

In [1]:
baseline_rule = {
    'f': ['e', 'ie', 'heit', 'ei', 'in', 'ik', 'keit', 'schaft', 'ung', 'tät', 'ur', 'tion'],
    'm': ['er', 'el', 'ling', 'ich', 'ig', 'ner', 'ismus', 'or', 'us', 'eich', 'ant'],
    'n': ['chen', 'o', 'lein', 'en', 'il', 'ma', 'tel', 'ment', 'nis', 'tum', 'um']
}

Here f stands for `feminin`, m stands for `maskulin`, and n stands for `neuter`.  Learning such rule is expected to take less effort than going through gender of many thousands of German nouns. But how well does such rule works? 

## Evaluation of the baseline rule

In order to evaluate the rule, a [database dump of en.wiktionary.org](https://dumps.wikimedia.org/enwiktionary/latest/) was parsed for German nouns. Script that does just that is located in [the "data" folder of this repository](https://gitlab.com/iaros/learn-german). Only the end result is attached to this notebook; Two versions are used - one with all the extracted nouns, and another with all the nouns that are also present in 150k phrases scraped from various subtitles to German shows. Loading procedures for the datasets are given below.

In [2]:
import json

# read the noun list
nouns = json.load(open('de_frequent_nouns.json', 'r'))

# simplify the list to the list of tuples
nouns = [(v['noun'], v['gender']) for v in nouns]

# weed out the p gender (apparently a plural)
nouns = [n for n in nouns if n[1] in {'f', 'm', 'n'}]

# example data
print('Total nouns:', len(nouns))
nouns[100:105]

Total nouns: 7619


[('Schatz', 'm'),
 ('Tür', 'f'),
 ('Bekommen', 'n'),
 ('Gestern', 'n'),
 ('Kerl', 'm')]

We will need evaluation procedure in later parts of this notebook, so lets write it down as reusable functions below. See comments in the code below for explanations; Note that this code is a bit more complex than what you would expect as it is reused later in the notebook.

In [3]:
import numpy as np

def rule_applies(rule, word):
    """Checks whether a rule given as a dict with keys f, m, n containing 
    values as arrays of ending applies to the word."""
    
    for endings in rule.values():
        for ending in endings:
            if word.endswith(ending):
                return True
    
    return False

def rule_predict(rule, word):
    # estimates ending for a given word
    result = 'f'  # default: feminine gender
    fitting_ending_length = 0  # longer ending takes precedence over shorter one
    
    for gender, endings in rule.items():
        for ending in endings:
            if word.endswith(ending) and len(ending) > fitting_ending_length:
                result = gender
                fitting_ending_length = len(ending)
    
    return result

def rule_confusion(rule):
    """Finds all endings that could be confused in the rule"""
    endings = [(e, g) for g in rule for e in rule[g]]
    
    all_confusion = []
    for end, gen in endings:
        confusion = []  # which endings can be confused?
        for end2, gen2 in endings:
            if end2.endswith(end) and len(end2) > len(end) and gen2 != gen:
                confusion.append(end2 + "(" + gen2 + ")")
        
        if confusion:
            all_confusion.append(end + "(" + gen + ") can be confused with " + ', '.join(confusion))
    
    return all_confusion

def rule_quality(rule, all_nouns, per_ending_accuracy=False, rule_analysis=False):
    """For a rule given as a dict with keys f, m, n containing 
    values as arrays of ending, evaluate this rule on a set of 
    nouns given as a list of tuples (noun str, gender \in m,f,n)
    
    Returns a dict with various rule performance metrics, such as:
    - Coverage: fraction of all supplied nouns, for which the rule
        applies;
    - Accuracy: overall accuracy of the rule on nouns where the
        rule applies. 
    """
    
    # result will be a dict with various metrics
    result = {}
    
    # first, select all the nouns that rule can be applied to
    nouns = [n for n in all_nouns if rule_applies(rule, n[0])]
    
    # percentage of all nouns covered by the rule
    result['Coverage %'] = round(len(nouns) / len(all_nouns), 3)
    result['Coverage count'] = len(nouns)
    result['Total count'] = len(all_nouns)
    
    # make estimations with rule
    y = np.array([gender for word, gender in nouns])  # ground truth
    y_pred = np.array([rule_predict(rule, word) for word, _ in nouns])  # estimations with rule
    # accuracy of the rule
    result['Accuracy'] = round(np.mean(y == y_pred), 3)
    
    # get the accuracy per class
    per_class_stats = {}
    for clas in set(y):
        I = y == clas
        per_class_stats[clas] = {
            'accuracy': round(np.mean(y[I] == y_pred[I]), 3),
            'instances': np.sum(I)
        }
    result['Gender accuracy'] = per_class_stats
    
    # get accuracy per ending of noun
    per_ending_stats = {}
    for gender, endings in rule.items():
        for ending in endings:
            I = np.array([word.endswith(ending) for word, gender in nouns])
            per_ending_stats[ending] = {
                'accuracy': round(np.mean(y[I] == y_pred[I]), 3),
                'gender': gender,
                'instances': np.sum(I)
            }
    
    result['Ending count'] = len(per_ending_stats)
    
    if per_ending_accuracy:
        result['Ending accuracy'] = per_ending_stats
    
    return result

Alright, now lets apply the functions to the data and the baseline rule!

In [4]:
from pprint import pprint
result = rule_quality(baseline_rule, nouns, True)
pprint(result, width=120)

{'Accuracy': 0.828,
 'Coverage %': 0.633,
 'Coverage count': 4821,
 'Ending accuracy': {'ant': {'accuracy': 0.944, 'gender': 'm', 'instances': 18},
                     'chen': {'accuracy': 0.951, 'gender': 'n', 'instances': 162},
                     'e': {'accuracy': 0.874, 'gender': 'f', 'instances': 1364},
                     'ei': {'accuracy': 0.771, 'gender': 'f', 'instances': 48},
                     'eich': {'accuracy': 0.667, 'gender': 'm', 'instances': 12},
                     'el': {'accuracy': 0.619, 'gender': 'm', 'instances': 265},
                     'en': {'accuracy': 0.807, 'gender': 'n', 'instances': 797},
                     'er': {'accuracy': 0.787, 'gender': 'm', 'instances': 830},
                     'heit': {'accuracy': 1.0, 'gender': 'f', 'instances': 70},
                     'ich': {'accuracy': 0.762, 'gender': 'm', 'instances': 21},
                     'ie': {'accuracy': 0.946, 'gender': 'f', 'instances': 111},
                     'ig': {'accuracy': 1

## Discussion of baseline rule

First of all, the rule applies to roughly 4800 nouns out of 7600. On the **63% of of all German nouns where rule applies**, the accuracy is 83%. Thus for the next noun that you recognize as one fitting to the rule, you have **83% chance to recognize correct the gender**. There are also **34 endings to learn** to use the rule.

It is evident also that for some genders the rule is more accurate; In particular the feminine gender, the most numerous one, is also the one where the rule achieves maximal 95% accuracy. The rule gives 70% accuracy on other genders. Thus you can be more **confident with feminine nouns**.

Also depending on an ending of a noun, accuracy can vary quite largely; Almost all 500 nouns ending with `ung` are feminine, yet 60 percent of 250 of nouns ending with `el` are masculine.  So if you intend to learn those, **learn first endings with high accuracy**.



## Is there a better rule?

A better rule could be the one which with the same number of endings to memorize and recognition accuracy, would cover more nouns. Alternatively, for same nouns coverage and rule number, it would be more accurate. Of course, one can also try to optimize the number of rules. 

With an assumption that there could be a rule which is in some sense better, lets outline a strategy for how to search for a rule. Firstly, all possible endings can be extracted from the list of nouns (our dataset); then, for every ending one can count how indicative the ending is of some specific gender.  Imagine that we took some hypothetical `ending1` and `ending2`, and counted instances of feminine, masculine and neuter nouns that end with an ending:

`ending1: {'f': 20, 'm': 10, 'n':70}`

`ending2: {'f': 23, 'm': 1, 'n':1}`

Clearly, `ending2` is more indicative of specific gender `f`.  The `ending1` is less accurate, but it covers more nouns. One way to construct new rule is to compose it of endings, which strike a ballance between how indicative they are of particular gender, and how many nouns they cover. For example, we could look for all endings that cover at least 50 nouns, and separate the most likely gender with at least 80% accuracy. Then one can use all such endings as a rule. A function which implements such approach is given below. See comments there for more detail.

In [5]:
def derive_rule(nouns, min_accuracy=0.8, min_nouns=50):
    # this will contain bins for every ending
    ending_bins = {}

    # make counts for all endings
    for word, gender in nouns:
        word = word.lower()  # ending might include first letter of word
        for ending_len in [1, 2, 3, 4, 5, 6]:
            ending = word[-ending_len:]
            if ending not in ending_bins:
                ending_bins[ending] = {'f':0, 'm':0, 'n':0}
            ending_bins[ending][gender] += 1
    
    # calculate statistics for every ending
    endings = []
    for ending, count in ending_bins.items(): 
        endings.append({
            "ending": ending,
            "nouns": sum(count.values()),
            "accuracy": max(count.values()) / sum(count.values()),
            "gender": max(count.keys(), key=count.get)
        })
    
    # filter out the endings
    endings = [
        e for e in endings if e['nouns'] >= min_nouns and e['accuracy'] >= min_accuracy
    ]
    
    # make a rule out of this
    rule = {
        gender: [stats['ending'] for stats in endings if stats['gender'] == gender]
        for gender in ['f', 'm', 'n']
    }
    
    # remove endings within one gender that contain other ending
    for gender, endings in rule.items():
        rule[gender] = [
            e for e in endings 
            if not any([  # look for any ending which is shorter and current one's ending
                True for e2 in endings if len(e2) < len(e) and e.endswith(e2)
            ])
        ]
    
    return rule

pprint(derive_rule(nouns, min_accuracy=0.8, min_nouns=50))

{'f': ['e', 'it', 'ng', 'aft', 'ion', 'rin'],
 'm': ['ner', 'ag', 'f', 'all', 'tz', 'ger', 'ler', 'her'],
 'n': ['en']}


Let's evaluate how well such rule performs!

In [6]:
rule = derive_rule(nouns, min_accuracy=0.8, min_nouns=50)
result = rule_quality(rule, nouns, per_ending_accuracy=False)
pprint(result, width=120)

{'Accuracy': 0.872,
 'Coverage %': 0.492,
 'Coverage count': 3750,
 'Ending count': 15,
 'Gender accuracy': {'f': {'accuracy': 0.995, 'instances': 2135},
                     'm': {'accuracy': 0.581, 'instances': 866},
                     'n': {'accuracy': 0.858, 'instances': 749}},
 'Total count': 7619}


With such approach we derived a rule directly from the data with 15 endings, which has a bit higher accuracy of 87%, and reduced coverage of 49%. Notice that **less than half of rules** compared to baseline rule is used. This indicates that there might be a possibility to improve on baseline. Lets try and do that. To do so, two parameters need to be adjusted - `min_accuracy` and `min_nouns`. Lets do this in the sections below!

Note that the optimization of such paramters is done by hand, but potentially one could even use some optimization algorithm, such as Bayesian Optimization. For simplicity, a function below is used for optimization.

In [7]:
def experiment(min_accuracy=0.8, min_nouns=50):
    # derive the rule itself
    rule = derive_rule(nouns, min_accuracy, min_nouns)
    
    print('Quality or rule:')
    pprint(rule_quality(rule, nouns, per_ending_accuracy=False), width=120)
    
    # print the rule
    print('Derived rule:')
    pprint(rule, compact=True)
    
    print('Any confusing endings:')
    pprint(rule_confusion(rule))
    
    print('Ending per gender:')
    pprint({g: len(v) for g, v in rule.items()})

## Result 1. Rule with smallest possible number of endings, with stats of baseline

It is good to see results of your learning soon, so initial small rule might be of interest. The rule that matches baseline with minimal number of endings is given below.

Note that derived rule uses **half of rules of baseline**, that is, only **17 endings need to be memorized**. The rule has equivalent accuracy and coverage.

In [8]:
experiment(0.71, 37)

Quality or rule:
{'Accuracy': 0.832,
 'Coverage %': 0.629,
 'Coverage count': 4796,
 'Ending count': 17,
 'Gender accuracy': {'f': {'accuracy': 0.943, 'instances': 2322},
                     'm': {'accuracy': 0.746, 'instances': 1470},
                     'n': {'accuracy': 0.701, 'instances': 1004}},
 'Total count': 7619}
Derived rule:
{'f': ['e', 'it', 'ng', 'nz', 'on', 'ft', 'rin'],
 'm': ['nn', 'r', 'll', 'ag', 'f', 'tz', 'ang'],
 'n': ['en', 'aus', 'rn']}
Any confusing endings:
['ng(f) can be confused with ang(m)']
Ending per gender:
{'f': 7, 'm': 7, 'n': 3}


## Result 2. Highest possible coverage with baseline accuracy and number of endings

Lets see if we can improve on baseline rule coverage. More nouns covered directly translates into less learning of German; Therefore optimization of coverage leads to direct savings of learning time.

The resulting rule is given below. The rule has **same number of endings and accuracy**, yet it achieves **9.2% higher coverage, or 708 more nouns**.

In [9]:
experiment(0.66, 27)

Quality or rule:
{'Accuracy': 0.82,
 'Coverage %': 0.726,
 'Coverage count': 5534,
 'Ending count': 34,
 'Gender accuracy': {'f': {'accuracy': 0.929, 'instances': 2456},
                     'm': {'accuracy': 0.788, 'instances': 1913},
                     'n': {'accuracy': 0.645, 'instances': 1165}},
 'Total count': 7619}
Derived rule:
{'f': ['e', 'g', 'it', 'ei', 'ät', 'nz', 'on', 'ik', 'ft', 'rin'],
 'm': ['nn', 'r', 'ss', 'll', 'nd', 'ag', 'f', 'tz', 'ing', 'ug', 'sch', 'kel',
       'b', 'rm', 'an', 'gel', 'ang', 'ist', 'kt'],
 'n': ['en', 'aus', 'ment', 'rn', 'nis']}
Any confusing endings:
['g(f) can be confused with ag(m), ing(m), ug(m), ang(m)']
Ending per gender:
{'f': 10, 'm': 19, 'n': 5}


## Result 3. Near maximal coverage with minimal number of endings

Here is a simple idea: if two of the genders can be estimated accurately, if another word comes and its ending is unknown, it is probably of the remaining third gender. This offers an opportunity for reduced amount of information to be learned.

Below is a rule just like that.

In [13]:
from skopt import Optimizer
from skopt.space import Real, Integer


class myobj():
    def __init__(self, config):
        """
        Contains:
        {
            "objective": {
                "name": name prop
                "weight": weight of objective
            }
            "constraints": [
            {'name': name of constraint, 'target': value should be less than metric}
            ]
        }
        """
        self.config = config
    
    
    def __call__(self, min_accuracy, min_nouns):
        rule = derive_rule(nouns, min_accuracy, min_nouns)
        rq = rule_quality(rule, nouns, per_ending_accuracy=False)
        rc = rule_confusion(rule)
        rq['Exceptions'] = len(rc)
        
        # Indicates complexity if most complex endings are not learned
        rq['complexity'] = sum(len(r) for g, r in rule.items()) - max(len(r) for g, r in rule.items())
        
        
        
        config = self.config
        obj = config['objective']
        metric = rq[obj['name']] * obj['weight']
        
        for constraint in config['constraints']:
            value = rq[constraint['name']]
            if 'leq' in constraint:
                violation = max(0, value - constraint['leq']) * constraint['weight']
            else:
                violation = max(0, constraint['geq'] - value) * constraint['weight']
            metric += violation

        return metric
        
obj = myobj({
    'objective': {'name': 'Ending count', 'weight': 1.0 / 37},
    'constraints': [
        {'name': 'Accuracy', 'geq': 0.828, 'weight': 10.0},
        {'name': 'Coverage %', 'geq': 0.633, 'weight': 10.0},
    ]
})

        
obj = myobj({
    'objective': {'name': 'Accuracy', 'weight': -1.0},
    'constraints': [
        {'name': 'Coverage %', 'geq': 0.8, 'weight': 4.0},
        {'name': 'Ending count', 'leq': 45, 'weight': 10.0},
    ]
})

from random import randint, uniform
from joblib import Parallel, delayed

P = [(amin, nmin) for amin in np.linspace(0.3, 0.9, 73) for nmin in range(5, 70)]
print("Total work:", len(P))
F = Parallel(n_jobs=-1, verbose=5)(delayed(obj)(*p) for p in P)
i = np.argmin(F)
experiment(*P[i])
print('Best result:', P[i], 'with objective', F[i])

Total work: 4745


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   27.1s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   39.8s
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed:   55.4s
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 2162 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 3026 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 3512 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | ela

Quality or rule:
{'Accuracy': 0.799,
 'Coverage %': 0.806,
 'Coverage count': 6142,
 'Ending count': 38,
 'Gender accuracy': {'f': {'accuracy': 0.915, 'instances': 2529},
                     'm': {'accuracy': 0.773, 'instances': 2210},
                     'n': {'accuracy': 0.631, 'instances': 1403}},
 'Total count': 7619}
Derived rule:
{'f': ['e', 'g', 'it', 'ei', 'ät', 'nz', 'on', 'ik', 'ft', 'ur', 'rin'],
 'm': ['ch', 'nn', 'r', 'st', 'ss', 'll', 'nd', 'ag', 'f', 'tz', 'ing', 'ug',
       'el', 'b', 'rm', 'an', 'ang', 'kt'],
 'n': ['en', 'aus', 'ld', 'o', 'ent', 'rn', 'nis', 'il', 'um']}
Any confusing endings:
['g(f) can be confused with ag(m), ing(m), ug(m), ang(m)',
 'r(m) can be confused with ur(f)']
Ending per gender:
{'f': 11, 'm': 18, 'n': 9}
Best result: (0.5916666666666668, 30) with objective -0.799


[Parallel(n_jobs=-1)]: Done 4745 out of 4745 | elapsed:  5.1min finished


In [11]:
# (0.7029066640547783, 42) = 83, 63, 16
# (0.5137855775715072, 9) = 0.80, 97, 107 (62)
experiment(0.5137855775715072, 9)  # 80, 80, 37???

Quality or rule:
{'Accuracy': 0.8,
 'Coverage %': 0.974,
 'Coverage count': 7424,
 'Ending count': 107,
 'Gender accuracy': {'f': {'accuracy': 0.902, 'instances': 2752},
                     'm': {'accuracy': 0.776, 'instances': 2787},
                     'n': {'accuracy': 0.684, 'instances': 1885}},
 'Total count': 7619}
Derived rule:
{'f': ['e', 'in', 'hr', 'au', 'g', 'ht', 'it', 'i', 'ät', 'nz', 'on', 'tat',
       'ik', 'art', 'ft', 'uer', 'ur', 'hrt', 'bahn', 'ra', 'ex'],
 'm': ['h', 's', 'nn', 'r', 'ein', 'st', 'l', 'den', 'eg', 'nd', 'ag', 'ck',
       'nt', 'ott', 'f', 'ame', 'z', 'ort', 'ing', 'nk', 'ug', 'hn', 'wagen',
       'ig', 'alt', 'at', 'arzt', 'rd', 'b', 'm', 'an', 'mut', 'p', 'ang',
       'sten', 'kt', 'itt', 'ton', 'rg', 'pen', 'bau'],
 'n': ['es', 'n', 'aus', 'al', 'os', 'ld', 'o', 'echt', 'id', 'land', 'immer',
       'ind', 'ück', 'ent', 'tt', 'och', 'hiff', 'wort', 'ende', 'sser', 'erz',
       'ert', 'iel', 'tel', 'oss', 'nis', 'lz', 'ma', 'ot', 'isch', 'buc

## Summary

A number of rules for German noun gender estimation were derived from the noun data, scraped from wiktionary.org. All of the rules operate on the endings of the nouns; A baseline rule from a popular online resource for language learning is introduced. It is shown that all the derived rules improve in some way over the baseline rule. The rules will also be available [here](http://bit.do/d3l) for anyone wanting to learn them and practice estimating the noun gender.